In [1]:
pip install redis

Note: you may need to restart the kernel to use updated packages.


In [21]:
import redis

r = redis.Redis(
  host='redis-13187.c323.us-east-1-2.ec2.redns.redis-cloud.com',
  port=13187,
  password='kowtJQye8J07wVlQJZQ7mKa8lvq8ySrp')

In [18]:
import uuid

# Generate a random session ID
session_id = str(uuid.uuid4())

In [8]:
pip install tavily-python


Note: you may need to restart the kernel to use updated packages.


In [24]:
try:
    r.ping()
    print("Connected to Redis Cloud successfully!")
except Exception as e:
    print(f"Redis connection error: {e}")

Connected to Redis Cloud successfully!


In [37]:
import redis
from tavily import TavilyClient

# Initialize Redis and Tavily client
r = redis.Redis(host='localhost', port=6379, db=0)
TAVILY_API_KEY = "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"
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def web_lookup_and_store(query, session_id, base_url):
    try:
        # Perform a web search using Tavily
        search_result = tavily_client.search(query)

        # Check if the response contains 'results'
        if not search_result or 'results' not in search_result:
            print("No results found or invalid response format.")
            return None

        # Extract the first result
        results = search_result['results']
        if len(results) == 0:
            print("No search results available.")
            return None

        first_result = results[0]
        url = first_result.get('url', 'Unknown URL')
        content = first_result.get('content', '')

        # Store the content and URL in Redis
        key = f"session:{session_id}"
        r.hset(key, "response", content)
        r.hset(key, "source_url", url)

        # Generate the "View Details" link using the provided base URL
        view_details_url = f"{base_url}details?session_id={session_id}"

        print(f"Stored response in Redis with key: {key}")
        print(f"Source URL: {url}")
        print(f"View Details URL: {view_details_url}")

        return view_details_url

    except Exception as e:
        print(f"Error during web lookup and storage: {str(e)}")
        return None


In [38]:
@app.route("/search", methods=["GET"])
def search():
    query = request.args.get("query")
    session_id = request.args.get("session_id")
    base_url = request.host_url
    view_details_url = web_lookup_and_store(query, session_id, base_url)
    return {"view_details_url": view_details_url}, 200


In [39]:
if __name__ == "__main__":
    session_id = "1234"
    query = "Latest updates on AI technology"
    base_url = "http://127.0.0.1:5000/"
    view_details_url = web_lookup_and_store(query, session_id, base_url)
    print(f"Generated View Details URL: {view_details_url}")


Stored response in Redis with key: session:1234
Source URL: https://www.technologyreview.com/2024/01/08/1085096/artificial-intelligence-generative-ai-chatgpt-open-ai-breakthrough-technologies
View Details URL: http://127.0.0.1:5000/details?session_id=1234
Generated View Details URL: http://127.0.0.1:5000/details?session_id=1234
